In [24]:
import pandas as pd
from datetime import  date, timedelta
import time
import unicodedata
import string
import requests
import mysql.connector
import os

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

from sshtunnel import SSHTunnelForwarder
import pymysql as db
import pandas as pd
import datetime
import sshtunnel
import os
# .env ファイルをロードして環境変数へ反映
from dotenv import load_dotenv
load_dotenv('.env')
# 環境変数を参照

def removal_text(text:str) -> str:
    text = unicodedata.normalize("NFKC", text)
    text = text.translate(str.maketrans( '', '',string.punctuation  + '！'+ '　'+ ' '+'・'+'～' + '‐'))
    return text

def post_line_text(message:str,token:str) -> None:
    url = "https://notify-api.line.me/api/notify"
    headers = {"Authorization" : "Bearer "+ token}
    payload = {"message" :  message}
    post = requests.post(url ,headers = headers ,params=payload) 

def post_line_text_and_image(message:str,image_path:str,token:str) -> None:
    url = "https://notify-api.line.me/api/notify"
    headers = {"Authorization" : "Bearer "+ token}
    payload = {"message" :  message}
    #imagesフォルダの中のgazo.jpg
    print('image_path',image_path)
    files = {"imageFile":open(image_path,'rb')}
    post = requests.post(url ,headers = headers ,params=payload,files=files) 

def insert_data_bulk(df:pd.DataFrame,cnx) -> None:
    insert_sql = f"""INSERT INTO {os.getenv('WORDPRESS_DB_TABLE')}(店舗名, 日付, Nのつく日, 都道府県, 機種名, 台番号, G数, 差枚, BB, RB,ART, BB確率, RB確率, ART確率, 合成確率) values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
    cur = cnx.cursor()
    cur.executemany(insert_sql, df.values.tolist())
    cnx.commit()
    print("Insert bulk data")

def delete_data(cnx,day:int) -> None:
    cursor = cnx.cursor()
    target_days_ago = datetime.date.today() - datetime.timedelta(days=day)
    target_days_ago_str = target_days_ago.strftime('%Y-%m-%d')
    target_days_ago_str
    sql = f"DELETE FROM {os.getenv('WORDPRESS_DB_TABLE')} WHERE 日付 < '{target_days_ago_str} 00:00:00';"
    cursor.execute(sql)
    cnx.commit()
    
def query_prefecture_parlar_data(cnx,prefecture_name:str , past_target_day_number:int) -> pd.DataFrame :
    past_target_day:date = datetime.date.today() - timedelta(days=past_target_day_number)
    str_past_target_day:str = past_target_day.strftime('%Y-%m-%d')
    # SQLを実行する
    #SELECT 引っ張ってきたい列名　FROM　テーブル名 WHERE 条件列 = 'ジャグ（文字列の完全一致）' " 
    column_name = [ '店舗名','日付','Nのつく日', '都道府県', '機種名', '台番号', 'G数', '差枚', \
                    'BB', 'RB', 'ART', 'BB確率','RB確率', 'ART確率', '合成確率', 'id']
    print(column_name)
    with cnx.cursor() as cursor:
        sql = f"SELECT * FROM {os.getenv('WORDPRESS_DB_TABLE')} WHERE 都道府県 = '{prefecture_name}' AND 日付 = '{str_past_target_day} 00:00:00' ;" #AND 機種名 = '主役は銭形3'
        cursor.execute(sql)
        # # Select結果を取り出す
        #results = cursor.fetchall()
        print(sql)
        past_target_date_prefecture_parlar_data_df = pd.DataFrame(data=cursor.fetchall(), index = None, columns = column_name)
    return past_target_date_prefecture_parlar_data_df

In [22]:
with sshtunnel.SSHTunnelForwarder(
    (os.getenv('SSH_USERNAME'), 10022), 
    ssh_username="pachislot777", 
    ssh_private_key_password=os.getenv('SSH_PRIVATE_KEY_PASSWORD'), 
    ssh_pkey=r"sercret\akasaka.key", 
    remote_bind_address=("mysql8055.xserver.jp", 3306 )
    ) as server:

    # SSH接続確認
    print(f"local bind port: {server.local_bind_port}")
    # データベース接続
    cnx = mysql.connector.connect(
        host="localhost", 
        port=server.local_bind_port, 
        user=os.getenv('WORDPRESS_DB_ID'), 
        password=os.getenv('DB_PASSWORD'), 
        database=os.getenv('WORDPRESS_DB_NAME'), 
        charset="utf8",
        use_pure=True
        )

    # 接続確認
    print(f"sql connection status: {cnx.is_connected()}")
    cursor = cnx.cursor()
    past_target_date_prefecture_parlar_data_df = query_prefecture_parlar_data(cnx,'神奈川県', 2)
    # 終了
    display(past_target_date_prefecture_parlar_data_df)
    cnx.close()

local bind port: 58170
sql connection status: True
['店舗名', '日付', 'Nのつく日', '都道府県', '機種名', '台番号', 'G数', '差枚', 'BB', 'RB', 'ART', 'BB確率', 'RB確率', 'ART確率', '合成確率', 'id']
SELECT * FROM parlar_data WHERE 都道府県 = '神奈川県' AND 日付 = '2023-04-04 00:00:00' ;


,店舗名,日付,Nのつく日,都道府県,機種名,台番号,G数,差枚,BB,RB,ART,BB確率,RB確率,ART確率,合成確率,id
0,123横浜西口店,2023-04-04,4,神奈川県,マイジャグラーV,586,399,0,1,1,0,1/399.0,1/399.0,1/0.0,1/199.5,50608
1,123横浜西口店,2023-04-04,4,神奈川県,マイジャグラーV,587,3496,0,13,8,0,1/268.9,1/437.0,1/0.0,1/166.5,50609
2,123横浜西口店,2023-04-04,4,神奈川県,マイジャグラーV,588,4241,0,21,4,0,1/202.0,1/1060.2,1/0.0,1/169.6,50610
3,123横浜西口店,2023-04-04,4,神奈川県,マイジャグラーV,589,995,0,2,1,0,1/497.5,1/995.0,1/0.0,1/331.7,50611
4,123横浜西口店,2023-04-04,4,神奈川県,マイジャグラーV,590,5847,0,20,20,0,1/292.4,1/292.4,1/0.0,1/146.2,50612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27296,金時鶴見店,2023-04-04,4,神奈川県,劇場版魔法少女まどか☆マギカ前編始まりの物語後編永遠の物語,555,1495,-1307,10,10,0,1/149.5,1/149.5,,1/74.8,77904
27297,金時鶴見店,2023-04-04,4,神奈川県,戦姫絶唱シンフォギア勇気の歌,556,1701,-159,16,6,0,1/106.3,1/283.5,,1/77.3,77905
27298,金時鶴見店,2023-04-04,4,神奈川県,モンスターハンターワールド,563,0,0,0,0,0,1/0.0,1/0.0,,1/0.0,77906
27299,金時鶴見店,2023-04-04,4,神奈川県,えとたま,564,0,0,0,0,0,1/0.0,1/0.0,,1/0.0,77907


In [1]:
print('ライブラリの読み込み完了')
try:
    from src.utils import *
except:
    from utils import *

ライブラリの読み込み完了
読み込み完了


In [2]:
class UtilsDataClass(Blog):
    def __init__(self):
        super().__init__()
    def another_method(self):
        print(f'b')

In [3]:
utilsdata = UtilsDataClass()

In [4]:
utilsdata.add_target_date(-2)

インスタンに日付:4月4日(火)など三つの変数が追加されました


In [5]:
dir(utilsdata)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'add_target_date',
 'another_method',
 'check_url',
 'create_main_text',
 'error_pledge_name_list',
 'generate_by_pledge_text',
 'generate_thumbnail',
 'get_post_list',
 'post_blog',
 'post_line',
 'target_date',
 'target_date_string_jp',
 'target_date_string_sql',
 'upload_image',
 'wp',
 'wp_tag_add',
 'wp_update_post']

In [6]:
vars(utilsdata)

{'error_pledge_name_list': [],
 'wp': <wordpress_xmlrpc.base.Client at 0x209d5829ac0>,
 'target_date': datetime.date(2023, 4, 4),
 'target_date_string_jp': '4月4日(火)',
 'target_date_string_sql': '2023-04-04'}

In [9]:
post_list : list[WordPressPost] = utilsdata.get_post_list()
for post in post_list:
    print(post)
    #break

【4/11(火)】埼玉・神奈川・千葉　パチンコスロットイベント取材まとめ【関東】
【三重県】4月10日(月) パチンコスロットイベント取材まとめ
【福井県】4月10日(月) パチンコスロットイベント取材まとめ
【石川県】4月10日(月) パチンコスロットイベント取材まとめ
【富山県】4月10日(月) パチンコスロットイベント取材まとめ
【新潟県】4月10日(月) パチンコスロットイベント取材まとめ
【岐阜県】4月10日(月) パチンコスロットイベント取材まとめ
【長野県】4月10日(月) パチンコスロットイベント取材まとめ
【山梨県】4月10日(月) パチンコスロットイベント取材まとめ
【静岡県】4月10日(月) パチンコスロットイベント取材まとめ
【愛知県】4月10日(月) パチンコスロットイベント取材まとめ
【石川県】4月5日(水) パチンコスロットイベント取材まとめ
【富山県】4月5日(水) パチンコスロットイベント取材まとめ
【新潟県】4月5日(水) パチンコスロットイベント取材まとめ
【岐阜県】4月5日(水) パチンコスロットイベント取材まとめ
【長野県】4月5日(水) パチンコスロットイベント取材まとめ
【山梨県】4月5日(水) パチンコスロットイベント取材まとめ
【静岡県】4月5日(水) パチンコスロットイベント取材まとめ
【愛知県】4月5日(水) パチンコスロットイベント取材まとめ
【4/5(水)】埼玉・神奈川・千葉　パチンコスロットイベント取材まとめ【関東】
【4/10(月)】埼玉・神奈川・千葉　パチンコスロットイベント取材まとめ【関東】
【三重県】4月9日(日) パチンコスロットイベント取材まとめ
【福井県】4月9日(日) パチンコスロットイベント取材まとめ
【石川県】4月9日(日) パチンコスロットイベント取材まとめ
【富山県】4月9日(日) パチンコスロットイベント取材まとめ
【新潟県】4月9日(日) パチンコスロットイベント取材まとめ
【岐阜県】4月9日(日) パチンコスロットイベント取材まとめ
【長野県】4月9日(日) パチンコスロットイベント取材まとめ
【山梨県】4月9日(日) パチンコスロットイベント取材まとめ
【静岡県】4月9日(日) パチンコスロットイベント取材まとめ
【三重県】4月8日(土) パチンコスロットイベント取材まとめ


In [ ]:
for target_day_number in range(0,6):
    scraping.add_target_date(target_day_number)
    browser = scraping.login_scraping_site('chubu')
    prefecture_name_and_number_dict = scraping.get_prefecture_name_and_number_dict()
    for prefecture_name in prefecture_name_and_number_dict:
        blog = Blog()
        print(prefecture_name,target_day_number)
        blog.add_target_date (target_day_number)
        blog.prefecture_name = prefecture_name
        print(prefecture_name)
        scraping_target_date_pledge_df = scraping.read_target_date_pledge_df(prefecture_name)
        read_convert_parlar_name_df = scraping.read_convert_parlar_name_df()
        merged_syuzai_pledge_df = scraping.generate_merged_syuzai_pledge_df()
        save_main_image_path_list = scraping.create_pledge_main_images()
        title = f"【{blog.prefecture_name}】{blog.target_date_string_jp } パチンコスロットイベント取材まとめ"
        if title in post_title_contentid_dict:
            update_content_id:int = int(post_title_contentid_dict[title])
            print('既存の記事を更新します',update_content_id)
            files:list[Client] = blog.wp.call(methods.media.GetMediaLibrary({"parent_id": update_content_id}))
            for file in files:
                print('画像削除',file.id, file.title)
                ret = blog.wp.call(methods.posts.DeletePost(file.id))
                print(ret)
                #break